In [1]:
ticker = "AMC"
TESTING = True

In [2]:
API_KEY =  '0MXZO7Y5VXKK2R3PXN5XH8EZEWJTG6BJ' + '@AMER.OAUTHAP'
ACCT_NUMBER = '255612401'
CALLBACK_URL = 'https://localhost:8888'


In [3]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random


from tda import auth, client
from tda.orders.equities import equity_buy_market, equity_sell_market
from tda.orders.common import Duration, Session
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt
from datetime import datetime
import json
import schedule
import time
from dateutil.relativedelta import relativedelta

sns.set()


In [4]:
import pkg_resources
import types

def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split('.')[0]
        elif isinstance(val, type):
            name = val.__module__.split('.')[0]
        poorly_named_packages = {'PIL': 'Pillow', 'sklearn': 'scikit-learn'}
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]
        yield name


imports = list(set(get_imports()))
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name != 'pip':
        requirements.append((m.project_name, m.version))

#for r in requirements:
#    print('{}=={}'.format(*r))

In [5]:
class Deep_Evolution_Strategy:

    inputs = None

    def __init__(
        self, weights, reward_function, population_size, sigma, learning_rate
    ):
        self.weights = weights
        self.reward_function = reward_function
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate

    def _get_weight_from_population(self, weights, population):
        weights_population = []
        for index, i in enumerate(population):
            jittered = self.sigma * i
            weights_population.append(weights[index] + jittered)
        return weights_population

    def get_weights(self):
        return self.weights

    def train(self, epoch = 100, print_every = 1):
        lasttime = time.time()
        for i in range(epoch):
            population = []
            rewards = np.zeros(self.population_size)
            for k in range(self.population_size):
                x = []
                for w in self.weights:
                    x.append(np.random.randn(*w.shape))
                population.append(x)
            for k in range(self.population_size):
                weights_population = self._get_weight_from_population(
                    self.weights, population[k]
                )
                rewards[k] = self.reward_function(weights_population)
            rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-7)
            for index, w in enumerate(self.weights):
                A = np.array([p[index] for p in population])
                self.weights[index] = (
                    w
                    + self.learning_rate
                    / (self.population_size * self.sigma)
                    * np.dot(A.T, rewards).T
                )
            if (i + 1) % print_every == 0:
                print(
                    'iter %d. reward: %f'
                    % (i + 1, self.reward_function(self.weights))
                )
        print('time taken to train:', time.time() - lasttime, 'seconds')


class Model:
    def __init__(self, input_size, layer_size, output_size):
        self.weights = [
            np.random.randn(input_size, layer_size),
            np.random.randn(layer_size, output_size),
            np.random.randn(1, layer_size),
        ]

    def predict(self, inputs):
        feed = np.dot(inputs, self.weights[0]) + self.weights[-1]
        decision = np.dot(feed, self.weights[1])
        return decision

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

In [6]:
class TDAmeritrade:
    
    def __init__(self, account_number, api_key, callback_url, symbol):
        token_path = 'token.pickle'
        self.SYMBOL = symbol
        self.API_KEY = api_key
        self.CALLBACK_URL = callback_url
        self.ACCOUNT_NUMBER = account_number
        try:
            self.c = auth.client_from_token_file(token_path, self.API_KEY)
        except FileNotFoundError:
            from selenium import webdriver
            with webdriver.Chrome(ChromeDriverManager().install()) as driver:
                self.c = auth.client_from_login_flow(
                    driver, self.API_KEY, self.CALLBACK_URL, token_path)

        return
    
    def get_prices(self, end):
        # fetch price history using tda-api
        r = self.c.get_price_history(self.SYMBOL,
                                period_type=client.Client.PriceHistory.PeriodType.DAY,
                                period=client.Client.PriceHistory.Period.THREE_DAYS,
                                frequency_type=client.Client.PriceHistory.FrequencyType.MINUTE,
                                frequency=client.Client.PriceHistory.Frequency.EVERY_THIRTY_MINUTES,
                                end_datetime=end,
                                need_extended_hours_data=False
                                )

        assert r.status_code == 200, r.raise_for_status()

        # parse json and get candles data
        y = r.json()
        y = y["candles"]
        y = json.dumps(y)
        df = pd.read_json(y)
        # drop last row
        df = df[:-1]

        return df
    
    def get_daily_price_history(self, symbol, start_date, end_date):
        r = self.c.get_price_history_every_day(symbol=symbol, start_datetime=start_date, end_datetime=end_date)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()
        y = y["candles"]
        y = json.dumps(y)
       
        df = pd.read_json(y)        
        return df

    def get_cur_price(self):
        r = self.c.get_quote(self.SYMBOL)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()
        price = y[self.SYMBOL]["lastPrice"]

        return price

    # get useful account info
    def get_account(self):
        r = self.c.get_account(self.ACCOUNT_NUMBER)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()

        return y

    # see if we currently have a position
    def get_available_funds(self, use_margin = True):
        y = self.get_account()
        if use_margin:
            return y['securitiesAccount']['currentBalances']['buyingPower']
        else:
            return y['securitiesAccount']['currentBalances']['buyingPowerNonMarginableTrade']
    


    def get_has_position(self):
        pos = client.Client.Account.Fields.POSITIONS
        r = self.c.get_account(self.ACCOUNT_NUMBER, fields=[pos])
        y = r.json()
        
        if "positions" in y["securitiesAccount"]:
            return True
        else:
            return False

    def get_positions(self, symbol):
        pos = client.Client.Account.Fields.POSITIONS
        r = self.c.get_account(self.ACCOUNT_NUMBER, fields=[pos])
        y = r.json()
        if "positions" in y["securitiesAccount"]:
            y = y["securitiesAccount"]["positions"]    

            # Filter python objects with list comprehensions
            output_dict = [x for x in y if x['instrument']['symbol'] == ticker]
            y = json.dumps(output_dict)

            df = pd.read_json(y)
            return df[['averagePrice','longQuantity']] 
            
        else:
            return []
        

    def place_order(self, order_type, shares):
        if order_type == 'buy':
            order_spec = equity_buy_market(self.SYMBOL, shares).set_session(
                Session.NORMAL).set_duration(Duration.DAY).build()
            self.c.place_order(self.ACCOUNT_NUMBER, order_spec)

        if order_type == 'sell':
            order_spec = equity_sell_market(self.SYMBOL, shares).set_session(
                Session.NORMAL).set_duration(Duration.DAY).build()
            self.c.place_order(self.ACCOUNT_NUMBER, order_spec)

    def get_transactions(self):
        r = self.c.get_transactions(self.ACCOUNT_NUMBER)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()
        
        return y


td_ameritrade = TDAmeritrade(ACCT_NUMBER, API_KEY, CALLBACK_URL, ticker)

In [7]:
class Agent:

    POPULATION_SIZE = 90 #15
    SIGMA = 0.1
    LEARNING_RATE = 0.03

    def __init__(self, model, window_size, trend, skip, initial_money, trading_api):
        self.model = model
        self.window_size = window_size
        self.half_window = window_size // 2
        self.trend = trend
        self.skip = skip
        self.initial_money = initial_money
        self.trading_api = trading_api
        self.es = Deep_Evolution_Strategy(
            self.model.get_weights(),
            self.get_reward,
            self.POPULATION_SIZE,
            self.SIGMA,
            self.LEARNING_RATE,
        )

    
    def act(self, sequence):
        decision = self.model.predict(np.array(sequence))
        return np.argmax(decision[0])
    
    def get_state(self, t):
        window_size = self.window_size + 1
        d = t - window_size + 1
        block = self.trend[d : t + 1] if d >= 0 else -d * [self.trend[0]] + self.trend[0 : t + 1]
        res = []
        for i in range(window_size - 1):
            res.append(block[i + 1] - block[i])
        return np.array([res])

    def get_reward(self, weights):
        initial_money = self.initial_money
        starting_money = initial_money
        self.model.weights = weights
        state = self.get_state(0)
        inventory = []
        quantity = 0
        for t in range(0, len(self.trend) - 1, self.skip):
            action = self.act(state)
            next_state = self.get_state(t + 1)
            
            if action == 1 and starting_money >= self.trend[t]:
                inventory.append(self.trend[t])
                starting_money -= self.trend[t]
                
            elif action == 2 and len(inventory):
                bought_price = inventory.pop(0)
                starting_money += self.trend[t]

            state = next_state
        return ((starting_money - initial_money) / initial_money) * 100

    def fit(self, iterations, checkpoint):
        self.es.train(iterations, print_every = checkpoint)

    def buy(self,  current_inventory, current_price, days_since_training):        
        state = self.get_state(days_since_training)
        starting_money = self.initial_money
        action = self.act(state)

        if action == 1 and self.initial_money >= current_price:
            if TESTING == False:
                print('WARNING!!!! SENDING REAL ORDER')
                self.trading_api.place_order('buy', 1)

            current_inventory.append(current_price)
            self.initial_money -= current_price
            print('buy 1 unit at price %f, total balance %f'% (current_price, self.initial_money))

        elif action == 2 and len(current_inventory):
            if TESTING == False:
                print('WARNING!!!! SENDING REAL ORDER')
                self.trading_api.place_order('sell', 1)
                
            bought_price = current_inventory.pop(0)
            self.initial_money += current_price#self.trend[t]
            try:
                invest = ((current_price - bought_price) / bought_price) * 100
            except:
                invest = 0
            print('sell 1 unit at price %f, investment %f %%, total balance %f,' % (current_price, invest, self.initial_money) )

        invest = ((self.initial_money - starting_money) / starting_money) * 100
        total_gains = self.initial_money - starting_money
        
        return current_inventory

In [8]:

current_date = datetime.today() - relativedelta(days=1)
past_date = current_date - relativedelta(years=2)
df = td_ameritrade.get_daily_price_history(ticker, past_date, current_date)

df.rename(columns = {'datetime':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume':'Volume'}, inplace = True)
testing = df.copy()

if TESTING == True:
    tot = len(df)
    test_count = int(tot * .1)
    train_count = int(tot * .9)
    df = df.head(train_count)
    testing = testing.tail(test_count)

In [9]:
close = df.Close.values.tolist()
window_size = 30
skip = 1

avail_funds = td_ameritrade.get_available_funds()
print("Available Funds: "  + str(avail_funds))
model = Model(input_size = window_size, layer_size = 500, output_size = 3)
agent = Agent(model = model, 
              window_size = window_size,
              trend = close,
              skip = skip,
              initial_money = avail_funds,
              trading_api = td_ameritrade)
agent.fit(iterations = 200, checkpoint = 5)

Available Funds: 158.68
iter 5. reward: 117.569952
iter 10. reward: 309.503403
iter 15. reward: 516.322158
iter 20. reward: 583.539198
iter 25. reward: 729.178220
iter 30. reward: 871.458281
iter 35. reward: 917.853542
iter 40. reward: 991.088984
iter 45. reward: 1024.558861
iter 50. reward: 1121.445677
iter 55. reward: 1151.386438


In [ ]:
def trade_if_ai_says_to():
    print('trade_if_ai_says_to')

    last_dt = df.iloc[-1]['Date']
    now = datetime.now()
    countdown = now - last_dt
    days_since_training_the_model = countdown.days

    inventory = []
    positions = td_ameritrade.get_positions(ticker)
    rg = positions.longQuantity.values[0]
    val = positions.averagePrice.values[0]
    for i in range(0, rg):
        inventory.append(val)

    cur_price = td_ameritrade.get_cur_price()
    
    print("Price: " + str(cur_price) + " Day: " + str(days_since_training_the_model))
    inv =  agent.buy(inventory, cur_price, days_since_training_the_model)
    return inv
    
def test_trade_if_ai_says_to(index, inventory):
    last_dt = df.iloc[-1]['Date']
    now_dt = testing.iloc[index]['Date']
    countdown = now_dt - last_dt
    days_since_training_the_model = countdown.days
    cur_price = testing.iloc[index]['Open']
    
    print("Price: " + str(cur_price) + " Day: " + str(days_since_training_the_model))
    inv =  agent.buy(inventory, cur_price, days_since_training_the_model)
    return inv

In [ ]:

# TODO: Update Historical prices every-day?
# TODO: Retrain Every day?
# TODO: Figure out if the market is open
avail_funds = td_ameritrade.get_available_funds()
print("Available Funds: "  + str(avail_funds))
if TESTING == False:
    schedule.every().monday.at('07:30').do(trade_if_ai_says_to)
    schedule.every().tuesday.at('07:30').do(trade_if_ai_says_to)
    schedule.every().wednesday.at('07:30').do(trade_if_ai_says_to)
    schedule.every().thursday.at('07:30').do(trade_if_ai_says_to)
    schedule.every().friday.at('07:30').do(trade_if_ai_says_to)

    while True:
       schedule.run_pending()
       time.sleep(1)

elif TESTING == True:
    inventory = []
    positions = td_ameritrade.get_positions(ticker) 
    rg = positions.longQuantity.values[0]
    val = float(positions.averagePrice.values[0])
    for i in range(0, rg):
        inventory.append(val)

    rg = len(testing)
    for i in range(0, rg - 1):
        print('--------Inventory Before Trade: ' + str(len(inventory)))
        inventory = test_trade_if_ai_says_to(i, inventory)
        print('--------Inventory After Trade: ' + str(len(inventory)))
        
price = td_ameritrade.get_cur_price()
value = price * len(inventory)
print('Inventory: ' + str(len(inventory)))
print('Current Value of Inventory: ' + str(value))

Available Funds: 75.65
--------Inventory Before Trade: 1
Price: 8.78 Day: 2
buy 1 unit at price 8.780000, total balance 66.870000
--------Inventory After Trade: 2
--------Inventory Before Trade: 2
Price: 8.65 Day: 6
buy 1 unit at price 8.650000, total balance 58.220000
--------Inventory After Trade: 3
--------Inventory Before Trade: 3
Price: 8.07 Day: 7
--------Inventory After Trade: 3
--------Inventory Before Trade: 3
Price: 8.3 Day: 8
--------Inventory After Trade: 3
--------Inventory Before Trade: 3
Price: 8.86 Day: 9
sell 1 unit at price 8.860000, investment 19.891746 %, total balance 67.080000,
--------Inventory After Trade: 2
--------Inventory Before Trade: 2
Price: 10.01 Day: 12
--------Inventory After Trade: 2
--------Inventory Before Trade: 2
Price: 9.63 Day: 13
buy 1 unit at price 9.630000, total balance 57.450000
--------Inventory After Trade: 3
--------Inventory Before Trade: 3
Price: 9.52 Day: 14
sell 1 unit at price 9.520000, investment 8.428246 %, total balance 66.970000

In [ ]:
inventory = td_ameritrade.get_positions(ticker)
print(inventory)
#fig = plt.figure(figsize = (15,5))
#plt.plot(close, color='r', lw=2.)
#plt.plot(close, '^', markersize=10, color='m', label = 'buying signal', markevery = states_buy)
#plt.plot(close, 'v', markersize=10, color='k', label = 'selling signal', markevery = states_sell)
#plt.title('total gains %f, total investment %f%%'%(total_gains, invest))
#plt.legend()
#plt.show()

   averagePrice  longQuantity
0          7.39             1
